In [2]:
import numpy as np


In [3]:
train_file = "./data/sign_mnist_train.csv"
test_file = "./data/sign_mnist_test.csv"

# Load data
train_data = np.loadtxt(train_file, delimiter=",", skiprows=1)
test_data = np.loadtxt(test_file, delimiter=",", skiprows=1)

# Split data into features and labels
y_train = train_data[:, 0]
X_train = train_data[:, 1:]
y_test = test_data[:, 0]
X_test = test_data[:, 1:]

# Normalize data
X_train = X_train / 255
X_test = X_test / 255

X_train
X_test

OSError: ./data/sign_mnist_train.csv not found.

In [1]:

from sklearn.linear_model import LogisticRegression
# Create models
ovr_model = LogisticRegression(multi_class="ovr", solver="liblinear")
softmax_model = LogisticRegression(multi_class="multinomial", solver="lbfgs")

# Train models
ovr_model.fit(X_train, y_train)
softmax_model.fit(X_train, y_train)

# Predict
y_ovr_pred = ovr_model.predict(X_test)
y_softmax_pred = softmax_model.predict(X_test)

# Evaluate
print("OVR")
print("Training set score: {:.3f}".format(ovr_model.score(X_train, y_train)))
print("Test set score: {:.3f}".format(ovr_model.score(X_test, y_test)))
print("\nSoftmax")
print("Training set score: {:.3f}".format(softmax_model.score(X_train, y_train)))
print("Test set score: {:.3f}".format(softmax_model.score(X_test, y_test)))


NameError: name 'X_train' is not defined